# Rag chatbot
Let's take the next step in our journey! We've explored LLMs, chatbots, and RAG. Now, it's time to put them all together to create a powerful tool: a RAG chain with memory.

---
## 1.&nbsp; Installations and Settings 🛠️
Except one item, this is the same as the last notebook. The only new item is a line to download a saved verion of the vector database created from Alice's Adventures in Wonderland. This saves us loading, splitting, and vectorising the book all over again.

In [1]:
%%bash
pip install -qqq -U langchain-huggingface
pip install -qqq -U langchain
pip install -qqq -U langchain-community
pip install -qqq -U faiss-cpu

# download saved vector database for Alice's Adventures in Wonderland
gdown --folder 1A8A9lhcUXUKRrtCe7rckMlQtgmfLZRQH

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.7/314.7 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.9/124.9 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.7/142.7 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 26.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 974.0/974.0 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 27.4 MB/s eta 0:00:00
Processing file 1h_lk4wTr12FAEaCS3eIJ4xsdcmnuIGmt index.faiss
Processing file 1O0Jz2Lx5cZdpQM7S5uw6Kx9_OLm5DuSQ index.pkl


Retrieving folder contents
Retrieving folder contents completed
Building directory structure
Building directory structure completed
Downloading...
From: https://drive.google.com/uc?id=1h_lk4wTr12FAEaCS3eIJ4xsdcmnuIGmt
To: /content/faiss_index/index.faiss
100%|██████████| 421k/421k [00:00<00:00, 110MB/s]
Downloading...
From: https://drive.google.com/uc?id=1O0Jz2Lx5cZdpQM7S5uw6Kx9_OLm5DuSQ
To: /content/faiss_index/index.pkl
100%|██████████| 216k/216k [00:00<00:00, 81.5MB/s]
Download completed


In [2]:
import os
from google.colab import userdata # we stored our access token as a colab secret

os.environ["HUGGINGFACEHUB_API_TOKEN"] = userdata.get('HF_TOKEN')

---
## 2.&nbsp; Setting up the chain 🔗
There are 2 new items in this code that you haven't seen before:
* the `output_key` parameter in [ConversationBufferMemory](https://api.python.langchain.com/en/latest/memory/langchain.memory.buffer.ConversationBufferMemory.html)
* [ConversationalRetrievalChain](https://api.python.langchain.com/en/latest/chains/langchain.chains.conversational_retrieval.base.ConversationalRetrievalChain.html#)

The `ConversationalRetrievalChain` is the LangChain chain for RAG with memory.

The `output_key` parameter is necessary if you want to include both `memory` and `return_source_documents` with `ConversationalRetrievalChain`. Without this parameter in `memory`, you'll get an error in `ConversationalRetrievalChain` when using both the aformentioned parameters as it gets two parameters when it's expecting one. Adding this parameter to `memory` means it knows which output to accept. If you're not using both `memory` and `return_source_documents` with `ConversationalRetrievalChain`, this isn't necessary.

In [3]:
from langchain_huggingface import HuggingFaceEndpoint, HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain_core.prompts import PromptTemplate
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain

hf_model = "mistralai/Mistral-7B-Instruct-v0.3"
llm = HuggingFaceEndpoint(repo_id=hf_model)

embedding_model = "sentence-transformers/all-MiniLM-l6-v2"
embeddings_folder = "/content/"

embeddings = HuggingFaceEmbeddings(model_name=embedding_model,
                                   cache_folder=embeddings_folder)

vector_db = FAISS.load_local("/content/faiss_index", embeddings, allow_dangerous_deserialization=True)

retriever = vector_db.as_retriever(search_kwargs={"k": 2})

memory = ConversationBufferMemory(memory_key = 'chat_history',
                                  return_messages = True,
                                  output_key = 'answer')  # Set output_key to 'answer'

template = """You are a nice chatbot having a conversation with a human. Answer the question based only on the following context and previous conversation. Keep your answers short and succinct.

Previous conversation:
{chat_history}

Context to answer question:
{context}

New human question: {question}
Response:"""

prompt = PromptTemplate(template = template,
                        input_variables = ["context", "question"])

# chain
chain = ConversationalRetrievalChain.from_llm(llm,
                                              retriever = retriever,
                                              memory = memory,
                                              return_source_documents = True,
                                              combine_docs_chain_kwargs = {"prompt": prompt})

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [4]:
chain.invoke("Who is the queen?")

{'question': 'Who is the queen?',
 'chat_history': [HumanMessage(content='Who is the queen?'),
  AIMessage(content=' The Queen in the context is the Queen of Hearts.')],
 'answer': ' The Queen in the context is the Queen of Hearts.',
 'source_documents': [Document(page_content='“Would you tell me,” said Alice, a little timidly, “why you are\npainting those roses?”\n\nFive and Seven said nothing, but looked at Two. Two began in a low\nvoice, “Why the fact is, you see, Miss, this here ought to have been a\n_red_ rose-tree, and we put a white one in by mistake; and if the Queen\nwas to find it out, we should all have our heads cut off, you know. So\nyou see, Miss, we’re doing our best, afore she comes, to—” At this\nmoment Five, who had been anxiously looking across the garden, called\nout “The Queen! The Queen!” and the three gardeners instantly threw\nthemselves flat upon their faces. There was a sound of many footsteps,\nand Alice looked round, eager to see the Queen.', metadata={'sour

In [5]:
print(chain.invoke("What does she enjoy doing?")["answer"])

 The Queen of Hearts enjoys making tarts.


In [6]:
print(chain.invoke("Whose head does she chop off?")["answer"])

 The Queen of Hearts threatens to chop off someone's head, but it's not shown in the context that she actually does it.


We can also use a while loop here to make our chatbot a little more interactive.

In [8]:
chain_2 = ConversationalRetrievalChain.from_llm(llm,
                                                retriever = retriever,
                                                memory = memory,
                                                return_source_documents = False,
                                                combine_docs_chain_kwargs = {"prompt": prompt})

# Start the conversation loop
while True:
  user_input = input("You: ")

  # Check for exit condition
  if user_input.lower() == 'end':
      print("Ending the conversation. Goodbye!")
      break

  # Get the response from the conversation chain
  response = chain_2.invoke(user_input)

  # Print the chatbot's response
  print("Chatbot:", response["answer"])

You: summarise our conversation so far
Chatbot:  Our conversation has been about the Queen of Hearts, who enjoys making tarts, and is involved in a trial in the story.
You: Who is on trial?
Chatbot:   The Queen of Hearts.
You: What did she do?
Chatbot:  The Queen of Hearts is on trial for having a tart stolen by the Knave of Hearts.
You: Shouldn't the knave be on trial if he stole?
Chatbot:  Yes, the Knave of Hearts is accused of stealing the tarts.
You: end
Ending the conversation. Goodbye!
